In [1]:
import cv2
import numpy as np
import os
import librosa 


import keras
from keras.applications import DenseNet121
from keras.models import Model, Sequential
from keras import layers



import tensorflow as tf
import tensorflow.keras.layers
from tensorflow.keras.preprocessing import image

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Input, Concatenate, Dense
from tensorflow.keras.models import Model


from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, LabelBinarizer



from PIL import Image


In [2]:
def compute_melgram(audio_path, new_shape=(256, 1296)):


    # Mel-spectrogram parameters
    SR = 12000
    N_FFT = 2048
    N_MELS = 256
    HOP_LEN = 256
    DURA = 27.64  # to make it 1296 frames

    # Load audio file
    src, sr = librosa.load(audio_path, sr=SR)

    # Ensure the audio is of desired duration
    n_sample = src.shape[0]
    n_sample_fit = int(DURA * SR)
    if n_sample < n_sample_fit:  # if too short
        src = np.hstack((src, np.zeros((int(DURA * SR) - n_sample,))))
    elif n_sample > n_sample_fit:  # if too long
        src = src[(n_sample - n_sample_fit) // 2:(n_sample + n_sample_fit) // 2]
    melgram = librosa.feature.melspectrogram(y=src, sr=SR, hop_length=HOP_LEN, n_fft=N_FFT, n_mels=N_MELS)

    return melgram
def compute_stft(audio_path,new_shape=(256, 1296)):


    # STFT parameters
    SR = 22115
    N_FFT = 510
    HOP_LEN = 512

    # Load audio file
    src, sr = librosa.load(audio_path, sr=SR)

    # Compute STFT
    stft = librosa.stft(src, n_fft=N_FFT, hop_length=HOP_LEN)

    return stft

def compute_mfcc(audio_path,new_shape=(256, 1296)):


    # MFCC parameters
    SR = 22115
    N_MFCC = 256
    N_FFT = 510
    HOP_LEN = 512

    # Load audio file
    src, sr = librosa.load(audio_path, sr=SR)

    # Compute MFCC
    mfcc = librosa.feature.mfcc(y=src, sr=SR, n_mfcc=N_MFCC, n_fft=N_FFT, hop_length=HOP_LEN)
    return mfcc


def process_audio_files(audio_dir):
    stft_data = []
    mel_data = []
    mfcc_data = []
    labels = []

    max_frames = 1296  # Set a maximum number of frames
    n_freq_bins = 256  # Number of frequency bins in the Mel-spectrogram

    for genre_folder in os.listdir(audio_dir):
        genre_path = os.path.join(audio_dir, genre_folder)
        if os.path.isdir(genre_path):
            for filename in os.listdir(genre_path):
                if filename.endswith('.wav'):
                    file_path = os.path.join(genre_path, filename)
                    stft = compute_stft(file_path)
                    mel = compute_melgram(file_path)
                    mfcc = compute_mfcc(file_path)
                    
                    # Upsample MFCCs
                    mfcc_upsampled = upsample_mfcc(mfcc, n_freq_bins)
                    
                    # Pad or crop arrays to ensure consistent shape
                    stft = pad_or_crop_array(stft, max_frames)
                    mel = pad_or_crop_array(mel, max_frames)
                    mfcc_upsampled = pad_or_crop_array(mfcc_upsampled, max_frames)
                    
                    stft_data.append(stft)
                    mel_data.append(mel)
                    mfcc_data.append(mfcc_upsampled)
                    labels.append(genre_folder)

    stft_data = np.array(stft_data)
    mel_data = np.array(mel_data)
    mfcc_data = np.array(mfcc_data)
    labels = np.array(labels)

    return stft_data, mel_data, mfcc_data, labels

# Function to upsample MFCCs
def upsample_mfcc(mfcc_data, n_freq_bins):
    n_mfcc, n_frames = mfcc_data.shape
    mfcc_upsampled = np.zeros((n_freq_bins, n_frames))

    # Compute interpolation factor for each MFCC coefficient
    factor = n_freq_bins // n_mfcc

    # Upsample each MFCC coefficient
    for i in range(n_mfcc):
        mfcc_upsampled[i*factor:(i+1)*factor, :] = mfcc_data[i]

    return mfcc_upsampled


def pad_or_crop_array(array, target_length):
    if array.shape[1] < target_length:
        # Pad array with zeros along the time axis
        padding = target_length - array.shape[1]
        array = np.pad(array, ((0, 0), (0, padding)), mode='constant')
    elif array.shape[1] > target_length:
        # Crop array along the time axis
        array = array[:, :target_length]
    return array

audio_dir = 'D:\ProjectMusicGenre\data\VNTM3'
stft_data, mel_data, mfcc_data, labels = process_audio_files(audio_dir)

In [3]:
# Here you can continue with your splitting and reshaping steps as before
stft_data = stft_data.reshape(stft_data.shape[0], stft_data.shape[1], stft_data.shape[2], 1)
mel_data = mel_data.reshape(mel_data.shape[0], mel_data.shape[1], mel_data.shape[2], 1)
mfcc_data = mfcc_data.reshape(mfcc_data.shape[0], mfcc_data.shape[1], mfcc_data.shape[2], 1)


In [4]:
stft_data=stft_data.astype("float32")

C:\Users\Admin\AppData\Local\Temp\ipykernel_10940\1535179258.py:1: ComplexWarning: Casting complex values to real discards the imaginary part
  stft_data=stft_data.astype("float32")


In [5]:
mel_data =  mel_data.astype("float32")

In [6]:
mfcc_data =mfcc_data.astype("float32")

In [7]:

# Split the data into training and testing sets (80% train, 20% test)
stft_train, stft_test, mel_train, mel_test, mfcc_train, mfcc_test, labels_train, labels_test = train_test_split(
    stft_data, mel_data, mfcc_data, labels, test_size=0.2, random_state=42)

# Split the training data into training and validation sets (80% train, 20% validation)
stft_test, stft_val, mel_test, mel_val, mfcc_test, mfcc_val, labels_test, labels_val = train_test_split(
    stft_test, mel_test, mfcc_test, labels_test, test_size=0.5, random_state=42)


In [8]:
checkpoint_filepath = r'D:\ProjectMusicGenre\data\checkpoint'
dataset_root =r'D:\ProjectMusicGenre\data'
saved_model_path = r'D:\ProjectMusicGenre\data\model'


In [9]:
from tensorflow.keras.layers import Dropout, Concatenate, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, BatchNormalization

def late_fusion(model,stft_input_shape, melgram_input_shape,mfcc_input_shape):
    # Define Group1 Block
    input_stft = Input(shape=stft_input_shape)
    stft = Conv2D(20, (7, 7), strides=(2, 2), activation='relu', padding='same')(input_stft)
    stft = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(stft)
    stft = dropout_block(stft, units=42)
    stft = transition_layer(stft, filters=42)
    stft = dropout_block(stft, units=85)
    stft = transition_layer(stft, filters=85)
    stft = dropout_block(stft, units=170)
    stft = transition_layer(stft, filters=170)
    stft = dropout_block(stft, units=341)
    stft = GlobalAveragePooling2D()(stft)  # Adding Global Average Pooling


    # Define Group2 Block
    input_melgram = Input(shape=melgram_input_shape)
    melgram = Conv2D(20, (7, 7), strides=(2, 2), activation='relu', padding='same')(input_melgram)
    melgram = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(melgram)
    melgram = dropout_block(melgram, units=42)
    melgram = transition_layer(melgram, filters=42)
    melgram = dropout_block(melgram, units=85)
    melgram = transition_layer(melgram, filters=85)
    melgram = dropout_block(melgram, units=170)
    melgram = transition_layer(melgram, filters=170)
    melgram = dropout_block(melgram, units=341)
    melgram = GlobalAveragePooling2D()(melgram)
    # Define Group3 Block


    input_mfcc = Input(shape=mfcc_input_shape)
    mfcc = Conv2D(20, (7, 7), strides=(2, 2), activation='relu', padding='same')(input_mfcc)
    mfcc = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(mfcc)
    mfcc = dropout_block(mfcc, units=42)  # Using a function for dropout block
    mfcc = transition_layer(mfcc, filters=42)
    mfcc = dropout_block(mfcc, units=85)
    mfcc = transition_layer(mfcc, filters=85)
    mfcc = dropout_block(mfcc, units=170)
    mfcc = transition_layer(mfcc, filters=170)
    mfcc = dropout_block(mfcc, units=341)
    mfcc = GlobalAveragePooling2D()(mfcc)  # Adding Global Average Pooling

    # Concatenate outputs of all groups
    concatenated_output = Concatenate()([ stft, melgram,mfcc])

    # Global Average Pooling
    # Fully Connected Layer
    output = Dense(5, activation='softmax')(concatenated_output)

    # Define the model
    model = Model(inputs=[ input_stft, input_melgram,input_mfcc], outputs=output)
    if not os.path.exists(checkpoint_filepath + '/latefusion'):
        os.makedirs(checkpoint_filepath + '/latefusion')
    checkpoint1= tf.keras.callbacks.ModelCheckpoint(
    filepath= checkpoint_filepath + '/latefusion' + '/latefusion_{epoch:02d}_{val_accuracy:.4f}.weights.h5',
    monitor='val_accuracy',
    save_best_only=True,
    save_weights_only=True,
    verbose=1
    )

    return model,checkpoint1

def dropout_block(input_layer, units):
    x = Dense(units, activation='relu')(input_layer)
    x = Dropout(0.1)(x)
    return x

def transition_layer(input_layer, filters):
    x = Conv2D(filters, (1, 1), activation='relu')(input_layer)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    return x


In [10]:
from tensorflow.keras.optimizers import Adam

# Instantiate the late fusion model using provided input shapes
mfcc_input_shape = (256, 1296, 1)  # Example shape, adjust according to your actual data
stft_input_shape = (256, 1296, 1)  # Example shape, adjust according to your actual data
melgram_input_shape = (256, 1296, 1)  # Example shape, adjust according to your actual data
model1 = tf.keras.Sequential()
model1,checkpoint1 = late_fusion(model1,stft_input_shape ,melgram_input_shape, mfcc_input_shape,)

# Compile the model
model1.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model1.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 256, 1296, │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 256, 1296, │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_2       │ (None, 256, 1296, │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 128, 648,  │      1,000 │ input_layer[0][0] │
│                     │ 20)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 128, 648,  │      1,000 │ input_layer_1[0]… │
│                     │ 20)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 128, 648,  │      1,000 │ input_layer_2[0]… │
│                     │ 20)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 63, 323,   │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │ 20)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_4     │ (None, 63, 323,   │          0 │ conv2d_4[0][0]    │
│ (MaxPooling2D)      │ 20)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_8     │ (None, 63, 323,   │          0 │ conv2d_8[0][0]    │
│ (MaxPooling2D)      │ 20)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 63, 323,   │        882 │ max_pooling2d[0]… │
│                     │ 42)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 63, 323,   │        882 │ max_pooling2d_4[… │
│                     │ 42)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 63, 323,   │        882 │ max_pooling2d_8[… │
│                     │ 42)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 63, 323,   │          0 │ dense[0][0]       │
│                     │ 42)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 63, 323,   │          0 │ dense_4[0][0]     │
│                     │ 42)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_8 (Dropout) │ (None, 63, 323,   │          0 │ dense_8[0][0]     │
│                     │ 42)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 63, 323,   │      1,806 │ dropout[0][0]     │
│                     │ 42)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 63, 323,   │      1,806 │ dropout_4[0][0] 

 Total params: 355,082 (1.35 MB)

 Trainable params: 355,082 (1.35 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
from keras.callbacks import EarlyStopping

early = EarlyStopping(monitor='loss',
    patience= 5,
    verbose= 0,
    mode='auto',
    baseline= None,
    restore_best_weights= True)


In [12]:
from sklearn.preprocessing import LabelBinarizer

label_binarizer = LabelBinarizer()
y_train_one_hot = label_binarizer.fit_transform(labels_train)
y_val_one_hot = label_binarizer.transform(labels_val)
y_eval_one_hot = label_binarizer.transform(labels_test)

In [13]:
history = model1.fit([stft_train,  mel_train,mfcc_train], y_train_one_hot, batch_size=32, epochs=100, validation_data=([stft_val,mel_val,mfcc_val], y_val_one_hot),callbacks=[checkpoint1, early])


Epoch 1/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2865 - loss: 1.9246
Epoch 1: val_accuracy improved from -inf to 0.56800, saving model to D:\ProjectMusicGenre\data\checkpoint/latefusion/latefusion_01_0.5680.weights.h5
63/63 ━━━━━━━━━━━━━━━━━━━━ 140s 2s/step - accuracy: 0.2880 - loss: 1.9176 - val_accuracy: 0.5680 - val_loss: 1.0915
Epoch 2/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6089 - loss: 0.9938
Epoch 2: val_accuracy improved from 0.56800 to 0.66000, saving model to D:\ProjectMusicGenre\data\checkpoint/latefusion/latefusion_02_0.6600.weights.h5
63/63 ━━━━━━━━━━━━━━━━━━━━ 123s 2s/step - accuracy: 0.6090 - loss: 0.9933 - val_accuracy: 0.6600 - val_loss: 0.9777
Epoch 3/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6743 - loss: 0.8561
Epoch 3: val_accuracy improved from 0.66000 to 0.69600, saving model to D:\ProjectMusicGenre\data\checkpoint/latefusion/latefusion_03_0.6960.weights.h5
63/63 ━━━━━━━━━━━━━━━━━━━━ 121s 2s/step - accuracy: 0.6748 - lo

In [14]:
from sklearn.metrics import accuracy_score
from keras.metrics import AUC, F1Score, Precision, Accuracy

# Evaluate the model
predict = model1.predict([stft_test,mel_test,mfcc_test])
predicted_labels = label_binarizer.inverse_transform(predict).tolist()

# loss_eval, accuracy_eval = model.evaluate(X_eval, y_eval_one_hot)

# print(f'Evaluation Loss: {loss_eval}, Evaluation Accuracy: {accuracy_eval}')
# Calculate AUC ROC
auc_roc = AUC()
auc_roc.update_state(y_eval_one_hot, predict)
print("AUC ROC:", auc_roc.result().numpy())

# Calculate F1 Score
f1 = F1Score()
f1.update_state(y_eval_one_hot, predict)
print("F1 Score:", np.mean(f1.result().numpy()))

# Calculate Precision
pre = Precision()
pre.update_state(y_eval_one_hot, predict)
print("Precision Score", pre.result().numpy())

# Accuracy Score
acc = accuracy_score(labels_test, predicted_labels)
print("Accuracy Score", acc)


8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 490ms/step
AUC ROC: 0.9972679
F1 Score: 0.9404189
Precision Score 0.94779116
Accuracy Score 0.944


In [13]:
history = model1.fit([stft_train,  mel_train,mfcc_train], y_train_one_hot, batch_size=32, epochs=100, validation_data=([stft_val,mel_val,mfcc_val], y_val_one_hot),callbacks=[checkpoint1, early])


Epoch 1/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.3767 - loss: 1.5690
Epoch 1: val_accuracy improved from -inf to 0.58000, saving model to D:\ProjectMusicGenre\data\checkpoint/latefusion/latefusion_01_0.5800.weights.h5
63/63 ━━━━━━━━━━━━━━━━━━━━ 158s 2s/step - accuracy: 0.3782 - loss: 1.5644 - val_accuracy: 0.5800 - val_loss: 0.9361
Epoch 2/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6734 - loss: 0.8545
Epoch 2: val_accuracy improved from 0.58000 to 0.74000, saving model to D:\ProjectMusicGenre\data\checkpoint/latefusion/latefusion_02_0.7400.weights.h5
63/63 ━━━━━━━━━━━━━━━━━━━━ 125s 2s/step - accuracy: 0.6737 - loss: 0.8540 - val_accuracy: 0.7400 - val_loss: 0.7989
Epoch 3/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7155 - loss: 0.7648
Epoch 3: val_accuracy improved from 0.74000 to 0.80400, saving model to D:\ProjectMusicGenre\data\checkpoint/latefusion/latefusion_03_0.8040.weights.h5
63/63 ━━━━━━━━━━━━━━━━━━━━ 121s 2s/step - accuracy: 0.7157 - lo

In [14]:
from sklearn.metrics import accuracy_score
from keras.metrics import AUC, F1Score, Precision, Accuracy

# Evaluate the model
predict = model1.predict([stft_test,mel_test,mfcc_test])
predicted_labels = label_binarizer.inverse_transform(predict).tolist()

# loss_eval, accuracy_eval = model.evaluate(X_eval, y_eval_one_hot)

# print(f'Evaluation Loss: {loss_eval}, Evaluation Accuracy: {accuracy_eval}')
# Calculate AUC ROC
auc_roc = AUC()
auc_roc.update_state(y_eval_one_hot, predict)
print("AUC ROC:", auc_roc.result().numpy())

# Calculate F1 Score
f1 = F1Score()
f1.update_state(y_eval_one_hot, predict)
print("F1 Score:", np.mean(f1.result().numpy()))

# Calculate Precision
pre = Precision()
pre.update_state(y_eval_one_hot, predict)
print("Precision Score", pre.result().numpy())

# Accuracy Score
acc = accuracy_score(labels_test, predicted_labels)
print("Accuracy Score", acc)


8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 465ms/step
AUC ROC: 0.99470806
F1 Score: 0.9334642
Precision Score 0.9430894
Accuracy Score 0.936


In [13]:
history = model1.fit([stft_train,  mel_train,mfcc_train], y_train_one_hot, batch_size=32, epochs=100, validation_data=([stft_val,mel_val,mfcc_val], y_val_one_hot),callbacks=[checkpoint1, early])


Epoch 1/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2835 - loss: 1.8902
Epoch 1: val_accuracy improved from -inf to 0.50000, saving model to D:\ProjectMusicGenre\data\checkpoint/latefusion/latefusion_01_0.5000.weights.h5
63/63 ━━━━━━━━━━━━━━━━━━━━ 187s 2s/step - accuracy: 0.2851 - loss: 1.8837 - val_accuracy: 0.5000 - val_loss: 1.2806
Epoch 2/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5869 - loss: 1.0206
Epoch 2: val_accuracy improved from 0.50000 to 0.67200, saving model to D:\ProjectMusicGenre\data\checkpoint/latefusion/latefusion_02_0.6720.weights.h5
63/63 ━━━━━━━━━━━━━━━━━━━━ 123s 2s/step - accuracy: 0.5871 - loss: 1.0197 - val_accuracy: 0.6720 - val_loss: 0.8963
Epoch 3/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6918 - loss: 0.8071
Epoch 3: val_accuracy improved from 0.67200 to 0.69200, saving model to D:\ProjectMusicGenre\data\checkpoint/latefusion/latefusion_03_0.6920.weights.h5
63/63 ━━━━━━━━━━━━━━━━━━━━ 124s 2s/step - accuracy: 0.6923 - lo

In [14]:
from sklearn.metrics import accuracy_score
from keras.metrics import AUC, F1Score, Precision, Accuracy

# Evaluate the model
predict = model1.predict([stft_test,mel_test,mfcc_test])
predicted_labels = label_binarizer.inverse_transform(predict).tolist()

# loss_eval, accuracy_eval = model.evaluate(X_eval, y_eval_one_hot)

# print(f'Evaluation Loss: {loss_eval}, Evaluation Accuracy: {accuracy_eval}')
# Calculate AUC ROC
auc_roc = AUC()
auc_roc.update_state(y_eval_one_hot, predict)
print("AUC ROC:", auc_roc.result().numpy())

# Calculate F1 Score
f1 = F1Score()
f1.update_state(y_eval_one_hot, predict)
print("F1 Score:", np.mean(f1.result().numpy()))

# Calculate Precision
pre = Precision()
pre.update_state(y_eval_one_hot, predict)
print("Precision Score", pre.result().numpy())

# Accuracy Score
acc = accuracy_score(labels_test, predicted_labels)
print("Accuracy Score", acc)


8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 462ms/step
AUC ROC: 0.997244
F1 Score: 0.945145
Precision Score 0.9518072
Accuracy Score 0.948


In [13]:
history = model1.fit([stft_train,  mel_train,mfcc_train], y_train_one_hot, batch_size=32, epochs=100, validation_data=([stft_val,mel_val,mfcc_val], y_val_one_hot),callbacks=[checkpoint1, early])


Epoch 1/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.3280 - loss: 1.9578
Epoch 1: val_accuracy improved from -inf to 0.49600, saving model to D:\ProjectMusicGenre\data\checkpoint/latefusion/latefusion_01_0.4960.weights.h5
63/63 ━━━━━━━━━━━━━━━━━━━━ 139s 2s/step - accuracy: 0.3289 - loss: 1.9510 - val_accuracy: 0.4960 - val_loss: 1.2054
Epoch 2/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5475 - loss: 1.0713
Epoch 2: val_accuracy improved from 0.49600 to 0.50800, saving model to D:\ProjectMusicGenre\data\checkpoint/latefusion/latefusion_02_0.5080.weights.h5
63/63 ━━━━━━━━━━━━━━━━━━━━ 122s 2s/step - accuracy: 0.5481 - loss: 1.0700 - val_accuracy: 0.5080 - val_loss: 1.1198
Epoch 3/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6697 - loss: 0.8452
Epoch 3: val_accuracy improved from 0.50800 to 0.76800, saving model to D:\ProjectMusicGenre\data\checkpoint/latefusion/latefusion_03_0.7680.weights.h5
63/63 ━━━━━━━━━━━━━━━━━━━━ 122s 2s/step - accuracy: 0.6700 - lo

In [14]:
from sklearn.metrics import accuracy_score
from keras.metrics import AUC, F1Score, Precision, Accuracy

# Evaluate the model
predict = model1.predict([stft_test,mel_test,mfcc_test])
predicted_labels = label_binarizer.inverse_transform(predict).tolist()

# loss_eval, accuracy_eval = model.evaluate(X_eval, y_eval_one_hot)

# print(f'Evaluation Loss: {loss_eval}, Evaluation Accuracy: {accuracy_eval}')
# Calculate AUC ROC
auc_roc = AUC()
auc_roc.update_state(y_eval_one_hot, predict)
print("AUC ROC:", auc_roc.result().numpy())

# Calculate F1 Score
f1 = F1Score()
f1.update_state(y_eval_one_hot, predict)
print("F1 Score:", np.mean(f1.result().numpy()))

# Calculate Precision
pre = Precision()
pre.update_state(y_eval_one_hot, predict)
print("Precision Score", pre.result().numpy())

# Accuracy Score
acc = accuracy_score(labels_test, predicted_labels)
print("Accuracy Score", acc)


8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 478ms/step
AUC ROC: 0.99853593
F1 Score: 0.9587166
Precision Score 0.96385545
Accuracy Score 0.96


In [15]:
for i in range(len(labels_test)):
    print(labels_test[i], predicted_labels[i])

cheo cheo
hatxam hatxam
cheo cheo
hatxam hatxam
cheo cheo
catru catru
cailuong cailuong
catru catru
cheo cheo
catru catru
catru catru
cheo cheo
hatxam hatxam
cheo cheo
cheo cheo
cheo cheo
cheo cheo
chauvan chauvan
cheo cheo
cheo cheo
hatxam hatxam
cheo cheo
chauvan chauvan
cheo cheo
cheo cheo
catru catru
cheo cheo
chauvan chauvan
cailuong cailuong
cheo cheo
cailuong cheo
chauvan chauvan
cheo cheo
catru catru
cheo cheo
catru catru
chauvan chauvan
catru catru
chauvan chauvan
cailuong cailuong
cheo cheo
chauvan chauvan
cailuong cailuong
chauvan chauvan
hatxam hatxam
chauvan chauvan
chauvan chauvan
cailuong cailuong
hatxam hatxam
cheo cheo
cheo cheo
chauvan chauvan
hatxam hatxam
chauvan chauvan
catru catru
hatxam hatxam
catru catru
cailuong cheo
catru catru
hatxam hatxam
hatxam hatxam
catru catru
hatxam hatxam
hatxam hatxam
cailuong cheo
catru catru
cheo cheo
cheo cheo
hatxam hatxam
hatxam hatxam
cheo cheo
cheo cheo
hatxam hatxam
chauvan chauvan
cailuong cailuong
cailuong cailuong
cheo che